# Markov Random Fields for Collaborative Filtering

This notebook provides an implementation in Python 2.7 of the algorithm outlined in the paper 
"[Markov Random Fields for Collaborative Filtering](https://arxiv.org/abs/1910.09645)" 
at the 33rd Conference on Neural Information Processing Systems (NeurIPS 2019), Vancouver, Canada.

For reproducibility, the experiments utilize publicly available [code](https://github.com/dawenl/vae_cf) for pre-processing three popular data-sets and for evaluating the learned model. That code accompanies the paper "[Variational Autoencoders for Collaborative Filtering](https://arxiv.org/abs/1802.05814)" by Dawen Liang et al. at The Web Conference 2018. While the code for the Movielens-20M data-set was made publicly available, the code for pre-processing the other two data-sets can easily be obtained by modifying their code as described in their paper.

The experiments in the paper (where an AWS instance with 64 GB RAM and 16 vCPUs was used) may be re-run by following these three steps:
- Step 1: Pre-processing the data (utilizing the publicly available [code](https://github.com/dawenl/vae_cf))
- Step 2: Learning the MRF (this code implements the new algorithm)
- Step 3: Evaluation (utilizing the publicly available [code](https://github.com/dawenl/vae_cf))

## Step 1: Pre-processing the data 

Utilizing the publicly available [code](https://github.com/dawenl/vae_cf), which is copied below (with kind permission of Dawen Liang):
 - run their cells 1-26 for data pre-processing 
      - note that importing matplotlib, seaborn, and tensorflow may not be necessary for our purposes here
 - run their cells 29-31 for loading the training data
 
Note that the following code is modified as to pre-process the [MSD data-set](https://labrosa.ee.columbia.edu/millionsong/tasteprofile). For pre-processing the [MovieLens-20M data-set](https://grouplens.org/datasets/movielens/20m/), see their original publicly-available [code](https://github.com/dawenl/vae_cf).

In [1]:
import os
import shutil
import sys

import numpy as np
from scipy import sparse

import pandas as pd

import bottleneck as bn

In [2]:
# change to the location of the data
DATA_DIR = '/my/data/directory/'

itemId='songId'   # for MSD data

In [3]:
raw_data = pd.read_csv(os.path.join(DATA_DIR, 'train_triplets.txt'), sep='\t', header=None, names=['userId', 'songId', 'playCount'])

### Data splitting procedure
- Select 50K users as heldout users, 50K users as validation users, and the rest of the users for training
- Use all the items from the training users as item set
- For each of both validation and test user, subsample 80% as fold-in data and the rest for prediction 

In [4]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

In [5]:
def filter_triplets(tp, min_uc=5, min_sc=0):
    # Only keep the triplets for items which were clicked on by at least min_sc users. 
    if min_sc > 0:
        itemcount = get_count(tp, itemId)
        tp = tp[tp[itemId].isin(itemcount.index[itemcount >= min_sc])]
    
    # Only keep the triplets for users who clicked on at least min_uc items
    # After doing this, some of the items will have less than min_uc users, but should only be a small proportion
    if min_uc > 0:
        usercount = get_count(tp, 'userId')
        tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]
    
    # Update both usercount and itemcount after filtering
    usercount, itemcount = get_count(tp, 'userId'), get_count(tp, itemId) 
    return tp, usercount, itemcount

In [6]:
raw_data, user_activity, item_popularity = filter_triplets(raw_data, min_uc=20, min_sc=200) # for MSD data

In [7]:
sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" % 
      (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

After filtering, there are 33633450 watching events from 571355 users and 41140 movies (sparsity: 0.143%)


In [8]:
unique_uid = user_activity.index

np.random.seed(98765)
idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]

In [9]:
# create train/validation/test users
n_users = unique_uid.size
n_heldout_users = 50000 # for MSD data

tr_users = unique_uid[:(n_users - n_heldout_users * 2)]
vd_users = unique_uid[(n_users - n_heldout_users * 2): (n_users - n_heldout_users)]
te_users = unique_uid[(n_users - n_heldout_users):]

In [10]:
train_plays = raw_data.loc[raw_data['userId'].isin(tr_users)]

In [11]:
unique_sid = pd.unique(train_plays[itemId])

In [12]:
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [13]:
pro_dir = os.path.join(DATA_DIR, 'pro_sg')

if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)

with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

In [14]:
def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('userId')
    tr_list, te_list = list(), list()

    np.random.seed(98765)

    for i, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        if i % 5000 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    
    return data_tr, data_te

In [15]:
vad_plays = raw_data.loc[raw_data['userId'].isin(vd_users)]
vad_plays = vad_plays.loc[vad_plays[itemId].isin(unique_sid)]

In [16]:
vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

0 users sampled
5000 users sampled
10000 users sampled
15000 users sampled
20000 users sampled
25000 users sampled
30000 users sampled
35000 users sampled
40000 users sampled
45000 users sampled


In [17]:
test_plays = raw_data.loc[raw_data['userId'].isin(te_users)]
test_plays = test_plays.loc[test_plays[itemId].isin(unique_sid)]

In [18]:
test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

0 users sampled
5000 users sampled
10000 users sampled
15000 users sampled
20000 users sampled
25000 users sampled
30000 users sampled
35000 users sampled
40000 users sampled
45000 users sampled


### Save the data into (user_index, item_index) format

In [19]:
def numerize(tp):
    uid = map(lambda x: profile2id[x], tp['userId'])
    sid = map(lambda x: show2id[x], tp[itemId])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [20]:
train_data = numerize(train_plays)
train_data.to_csv(os.path.join(pro_dir, 'train.csv'), index=False)

In [21]:
vad_data_tr = numerize(vad_plays_tr)
vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr.csv'), index=False)

In [22]:
vad_data_te = numerize(vad_plays_te)
vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te.csv'), index=False)

In [23]:
test_data_tr = numerize(test_plays_tr)
test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr.csv'), index=False)

In [24]:
test_data_te = numerize(test_plays_te)
test_data_te.to_csv(os.path.join(pro_dir, 'test_te.csv'), index=False)

### Load the pre-processed training and validation data

In [25]:
unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

In [26]:
def load_train_data(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

In [27]:
train_data = load_train_data(os.path.join(pro_dir, 'train.csv'))

## Step 2: Learning the MRF model (implementation of the new algorithm)
Now run the following code and choose to learn 
- either the dense MRF model 
- or the sparse MRF model 

In [28]:
import time
from copy import deepcopy

In [29]:
class MyClock:
    startTime = time.time()
    def tic(self):
        self.startTime = time.time()
    def toc(self):
        secs = time.time() - self.startTime 
        print("... elapsed time: {} min {} sec".format(int(secs//60), secs%60) )

myClock = MyClock()
totalClock = MyClock()

### Pre-computation of the training data

In [30]:
# parameter for pre-computation of the training data (see the paper for details): 
#   exponent for re-scaling of the training data 
#   if alpha=1, then XtX becomes the correlation matrix below
alpha = 0.75  

In [31]:
# input: train_data ... user-item interaction matrix (sparse, binary), gets loaded in step 1 above.
userCount = train_data.shape[0]
XtX= np.asarray(train_data.T.dot(train_data).todense(), dtype = np.float32)  
del train_data  # only the item-item data-matrix XtX is needed in the following

mu=np.diag(XtX) / userCount   # the mean of the columns in train_data (for binary train_data)
variance_times_userCount = np.diag(XtX) - mu * mu * userCount # variances of columns in train_data (scaled by userCount)

# standardizing the data-matrix XtX (if alpha=1, then XtX becomes the correlation matrix)
XtX -= mu[:,None] *(mu* userCount)
rescaling = np.power(variance_times_userCount, alpha / 2.0) 
scaling = 1.0  / rescaling
XtX = scaling[:,None] * XtX * scaling

XtXdiag = deepcopy(np.diag(XtX))   
ii_diag = np.diag_indices(XtX.shape[0])

print("number of items: {}".format(len(mu)))
print("number of users: {}".format(userCount))




number of items: 41140
number of users: 471355


### Dense MRF model

In [32]:
def dense_solution(GG):  # input: GG ... XtX plus the L2-norm regularization on the diagonal
    BB=np.linalg.inv(GG)
    BB/=-np.diag(BB)
    BB[ii_diag]=0.0
    return BB

training the dense model:

In [33]:
# choose the L2-norm regularization for the data-set:
#L2reg = 6.0   # for ML-20M data
#L2reg = 6.0   # for Netflix data 
L2reg = 3.0    # for MSD data

print("training the dense model ...")
myClock.tic()
XtX[ii_diag] = XtXdiag + L2reg 
BB = dense_solution(XtX)
myClock.toc()

print("re-scaling BB back to the original item-popularities ...")
# and assuming that mu.T.dot(BB) == mu.T, see Appendix in the paper
myClock.tic()
BB = scaling[:,None] * BB * rescaling
myClock.toc()
BB = BB.astype(np.float32)  

training the dense model ...
... elapsed time: 15 min 24.0481009483 sec
re-scaling BB back to the original item-popularities ...
... elapsed time: 0 min 11.953250885 sec


### Sparse MRF model

In [34]:
def calculate_sparsity_pattern(XtX, threshold, maxInColumn):
    # this implements section 3.1 in the paper.
    
    print("sparsifying the data-matrix (section 3.1 in the paper) ...")
    myClock.tic()
    # apply threshold
    ix = np.where( np.abs(XtX) > threshold)
    AA= sparse.csc_matrix( (XtX[ix], ix), shape=XtX.shape, dtype=np.float32)
    # enforce maxInColumn, see section 3.1 in paper
    countInColumns=AA.getnnz(axis=0)
    iiList = np.where(countInColumns > maxInColumn)[0]
    print("    number of items with more than {} entries in column: {}".format(maxInColumn, len(iiList)) )
    for ii in iiList:
        jj= AA[:,ii].nonzero()[0]
        kk = bn.argpartition(-np.abs(np.asarray(AA[jj,ii].todense()).flatten()), maxInColumn)[maxInColumn:]
        AA[  jj[kk], ii ] = 0.0
    AA.eliminate_zeros()
    print("    resulting sparsity of AA: {}".format( AA.nnz*1.0 / AA.shape[0] / AA.shape[0]) )
    myClock.toc()

    return AA 

In [35]:
def sparse_parameter_estimation(rr, XtX, AA):
    # this implements section 3.2 in the paper

    # list L in the paper, sorted by item-counts per column, ties broken by item-popularities as reflected by np.diag(XtX)
    AAcountInColumns = AA.getnnz(axis=0)
    sortedList=np.argsort(AAcountInColumns+ np.diag(XtX) /2.0/ np.max(np.diag(XtX))   )[::-1]  

    print("iterating through steps 1,2, and 4 in section 3.2 of the paper ...")
    myClock.tic()
    todoIndicators=np.ones(AAcountInColumns.shape[0])
    blockList=[]   # list of blocks. Each block is a list of item-indices, to be processed in step 3 of the paper
    for ii in sortedList:
        if todoIndicators[ii]==1:
            nn, _, vals=sparse.find(AA[:,ii])  # step 1 in paper: set nn contains item ii and its neighbors N
            kk=np.argsort(np.abs(vals))[::-1]
            nn=nn[kk]
            blockList.append(nn) # list of items in the block, to be processed in step 3 below
            # remove possibly several items from list L, as determined by parameter rr (r in the paper) 
            dd_count=max(1,int(np.ceil(len(nn)*rr)))
            dd=nn[:dd_count] # set D, see step 2 in the paper
            todoIndicators[dd]=0  # step 4 in the paper        
    myClock.toc()

    print("now step 3 in section 3.2 of the paper: iterating ...")
    # now the (possibly heavy) computations of step 3:
    # given that steps 1,2,4 are already done, the following for-loop could be implemented in parallel.   
    myClock.tic()
    BBlist_ix1, BBlist_ix2, BBlist_val = [], [], []
    for nn in blockList:
        #calculate dense solution for the items in set nn
        BBblock=np.linalg.inv( XtX[np.ix_(nn,nn)] )
        BBblock/=-np.diag(BBblock)
        # determine set D based on parameter rr (r in the paper) 
        dd_count=max(1,int(np.ceil(len(nn)*rr)))
        dd=nn[:dd_count] # set D in paper
        # store the solution regarding the items in D
        blockix = np.meshgrid(dd,nn)
        BBlist_ix1.extend(blockix[1].flatten().tolist())
        BBlist_ix2.extend(blockix[0].flatten().tolist())
        BBlist_val.extend(BBblock[:,:dd_count].flatten().tolist())
    myClock.toc()

    print("final step: obtaining the sparse matrix BB by averaging the solutions regarding the various sets D ...")
    myClock.tic()
    BBsum = sparse.csc_matrix( (BBlist_val,  (BBlist_ix1, BBlist_ix2  )  ), shape=XtX.shape, dtype=np.float32) 
    BBcnt = sparse.csc_matrix( (np.ones(len(BBlist_ix1), dtype=np.float32),  (BBlist_ix1,BBlist_ix2  )  ), shape=XtX.shape, dtype=np.float32) 
    b_div= sparse.find(BBcnt)[2]
    b_3= sparse.find(BBsum)
    BBavg = sparse.csc_matrix( ( b_3[2] / b_div   ,  (b_3[0],b_3[1]  )  ), shape=XtX.shape, dtype=np.float32)
    BBavg[ii_diag]=0.0
    myClock.toc()

    print("forcing the sparsity pattern of AA onto BB ...")
    myClock.tic()
    BBavg = sparse.csr_matrix( ( np.asarray(BBavg[AA.nonzero()]).flatten(),  AA.nonzero() ), shape=BBavg.shape, dtype=np.float32)
    
    print("    resulting sparsity of learned BB: {}".format( BBavg.nnz * 1.0 / AA.shape[0] / AA.shape[0]) )
    myClock.toc()

    return BBavg

In [36]:
def sparse_solution(rr, threshold, maxInColumn, L2reg):
    
    # sparsity pattern, see section 3.1 in the paper
    XtX[ii_diag] = XtXdiag  
    AA = calculate_sparsity_pattern(XtX, threshold, maxInColumn)

    # parameter-estimation, see section 3.2 in the paper 
    XtX[ii_diag] = XtXdiag+L2reg 
    BBsparse = sparse_parameter_estimation(rr, XtX, AA)
    
    return BBsparse

training the sparse model:

In [37]:
# parameters for the sparse solution of the MSD data: 
# choose a sparsity level
threshold = 0.375    # results in sparsity 0.1 % (for alpha=0.75)
#threshold = 0.11    # results in sparsity 0.5 % (for alpha=0.75)
maxInColumn = 1000
# hyper-parameter r in the paper, which determines the trade-off between approximation-accuracy and training-time
rr = 0.5
# L2 norm regularization
L2reg = 1.0 

print("training the sparse model:\n")
totalClock.tic()
BBsparse = sparse_solution(rr, threshold, maxInColumn, L2reg)
print("\ntotal training time (including the time for determining the sparsity-pattern):")
totalClock.toc()

print("\nre-scaling BB back to the original item-popularities ...")
# assuming that mu.T.dot(BB) == mu, see Appendix in paper
myClock.tic()
BBsparse=sparse.diags(scaling).dot(BBsparse).dot(sparse.diags(rescaling))
myClock.toc()

print("\nfor the evaluation below: converting the sparse model into a dense-matrix-representation ...")
myClock.tic()
BB = np.asarray(BBsparse.todense(), dtype=np.float32) 
myClock.toc()

training the sparse model:

sparsifying the data-matrix (section 3.1 in the paper) ...
    number of items with more than 1000 entries in column: 0
    resulting sparsity of AA: 0.0010084055559
... elapsed time: 0 min 14.1621689796 sec
iterating through steps 1,2, and 4 in section 3.2 of the paper ...
... elapsed time: 0 min 5.203758955 sec
now step 3 in section 3.2 of the paper: iterating ...
... elapsed time: 0 min 11.6959278584 sec
final step: obtaining the sparse matrix BB by averaging the solutions regarding the various sets D ...
... elapsed time: 0 min 6.03114891052 sec
forcing the sparsity pattern of AA onto BB ...
    resulting sparsity of learned BB: 0.0010084055559
... elapsed time: 0 min 0.5450091362 sec

total training time (including the time for determining the sparsity-pattern):
... elapsed time: 0 min 37.8255307674 sec

re-scaling BB back to the original item-popularities ...
... elapsed time: 0 min 0.0728540420532 sec

for the evaluation below: converting the sparse m

## Step 3: Evaluating the MRF model 

Utilizing the publicly available [code](https://github.com/dawenl/vae_cf), which is copied below (with kind permission of Dawen Liang):

 - run their cell 32 for loading the test data
 - run their cells 35 and 36 for the ranking metrics (for later use in evaluation)
 - run their cells 45 and 46
 - modify and run their cell 50:
    - remove 2 lines: the one that starts with ```with``` and the line below
    - remove the indentation of the line that starts with ```for```
    - modify the line that starts with ```pred_val``` as follows: ```pred_val = X.dot(BB)```
        
 - run their cell 51


In [38]:
def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

In [39]:
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG

In [40]:
def Recall_at_k_batch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

### Load the test data and compute test metrics

In [41]:
test_data_tr, test_data_te = load_tr_te_data(
    os.path.join(pro_dir, 'test_tr.csv'),
    os.path.join(pro_dir, 'test_te.csv'))

In [42]:
N_test = test_data_tr.shape[0]
idxlist_test = range(N_test)

batch_size_test = 2000

In [43]:
n100_list, r20_list, r50_list = [], [], []


for bnum, st_idx in enumerate(range(0, N_test, batch_size_test)):
    end_idx = min(st_idx + batch_size_test, N_test)
    X = test_data_tr[idxlist_test[st_idx:end_idx]]

    if sparse.isspmatrix(X):
        X = X.toarray()
    X = X.astype('float32')

    pred_val = X.dot(BB)
    # exclude examples from training and validation (if any)
    pred_val[X.nonzero()] = -np.inf
    n100_list.append(NDCG_binary_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=100))
    r20_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=20))
    r50_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=50))

n100_list = np.concatenate(n100_list)
r20_list = np.concatenate(r20_list)
r50_list = np.concatenate(r50_list)

In [44]:
print("Test NDCG@100=%.5f (%.5f)" % (np.mean(n100_list), np.std(n100_list) / np.sqrt(len(n100_list))))
print("Test Recall@20=%.5f (%.5f)" % (np.mean(r20_list), np.std(r20_list) / np.sqrt(len(r20_list))))
print("Test Recall@50=%.5f (%.5f)" % (np.mean(r50_list), np.std(r50_list) / np.sqrt(len(r50_list))))

Test NDCG@100=0.38087 (0.00110)
Test Recall@20=0.32709 (0.00113)
Test Recall@50=0.41652 (0.00119)


... accuracy of the sparse approximation (with sparsity 0.1% and parameter r=0.5)